In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense,LSTM, Dropout
import math

In [4]:
df = pd.read_csv('all-stocks-5yr.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'all-stocks-5yr.csv'

In [ ]:
z = df.loc[df['Name'] == 'ZTS']
z.info()

In [ ]:
treino = z.iloc[:, 4:5].values
treino

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Criando um objeto MinMaxScaler
sc = MinMaxScaler()

# Aplicando a transformação de escala nos dados de treino
sc1 = sc.fit_transform(treino)

In [ ]:
x_treino = []  # Lista para armazenar as sequências de entrada
y_treino = []  # Lista para armazenar os valores de saída correspondentes
timestamp = 45  # Tamanho da janela de tempo deslizante
length = len(treino)  # Comprimento dos dados de treino

# Criando as sequências de entrada (x_treino) e os valores de saída (y_treino)
for i in range(timestamp, length):
    # Cria uma sequência de entrada com base na janela de tempo deslizante
    # A sequência contém os valores de sc1 a partir de i - timestamp até i (exclusivo)
    x_treino.append(sc1[i - timestamp:i, 0])
    
    # O valor de saída correspondente é o valor de sc1 no índice i
    y_treino.append(sc1[i, 0])

# Converte as listas em arrays numpy
x_treino = np.array(x_treino)
y_treino = np.array(y_treino)

x_treino

In [ ]:
x_treino = np.reshape(x_treino, (x_treino.shape[0], x_treino.shape[1], 1))
x_treino.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

# Inicializa um modelo sequencial
model = Sequential()

# Adiciona uma camada LSTM com 120 unidades, retornando sequências completas.
# A entrada é uma janela de tempo de forma (tamanho_da_janela_de_tempo, 1)
model.add(LSTM(units=120, return_sequences=True, input_shape=(x_treino.shape[1], 1)))
model.add(Dropout(0.2))  # Adiciona uma camada de dropout para regularização

# Adiciona mais uma camada LSTM com 120 unidades e dropout
model.add(LSTM(units=120, return_sequences=True))
model.add(Dropout(0.2))

# Adiciona mais uma camada LSTM com 120 unidades e dropout
model.add(LSTM(units=120, return_sequences=True))
model.add(Dropout(0.2))

# Adiciona uma camada LSTM final com 120 unidades, retornando apenas o último valor
model.add(LSTM(units=120, return_sequences=False))
model.add(Dropout(0.2))

# Adiciona uma camada densa com 1 unidade de saída para finalizar a previsão
model.add(Dense(units=1))

# Compila o modelo com otimizador 'adam' e função de perda 'mean_squared_error'
model.compile(optimizer='adam', loss='mean_squared_error')

# Imprime um resumo da arquitetura do modelo
model.summary()

In [ ]:
model.fit(x_treino, y_treino, epochs = 100, batch_size = 32)

In [ ]:
# Selecionando os dados específicos do DataFrame df onde a coluna 'Name' é igual a 'BA'
teste = df[df['Name'] == 'BA']

# Selecionando somente a coluna 'close' do DataFrame teste
teste = teste.loc[:, ['close']]

# Definindo o número de lags para usar como timestamp
timestamp = 45

# Criando a variável y_treino usando os valores de 'close' a partir do índice 'timestamp'
y_treino = teste['close'].iloc[timestamp:].values

In [ ]:
clossing_price = teste.iloc[:,0:].values
clossing_price_scaled = sc.transform(clossing_price)

In [ ]:
# Inicializando uma lista vazia para armazenar os dados de teste
x_teste = []

# Obtendo o comprimento dos dados de teste
length = len(teste)

# Iterando sobre os dados de teste
for i in range(timestamp, length):
    # Adicionando os dados escalados de fechamento à lista x_teste
    x_teste.append(clossing_price_scaled[i - timestamp:i, 0])

# Convertendo a lista para um array numpy
x_teste = np.array(x_teste)

# Redimensionando os dados de teste para o formato adequado para a entrada do modelo LSTM
x_teste = x_teste.reshape(x_teste.shape[0], x_teste.shape[1], 1)

In [ ]:
# Realizando a previsão dos preços das ações usando o modelo treinado
y_predict = model.predict(x_teste)

# Invertendo a transformação para obter os valores originais dos preços
predict_price = sc.inverse_transform(y_predict)

In [ ]:
# Importando a biblioteca necessária para visualização
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 15, 6

# Plot dos preços reais das ações (em azul) e preços previstos (em vermelho)
plt.plot(y_treino, color='b', label='Preço atual das ações')
plt.plot(predict_price, color='r', label='Preço previsto das ações')

# Adicionando título e rótulos dos eixos
plt.title('Previsão do Preço das Ações')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')

# Adicionando legenda para as linhas
plt.legend()

# Exibindo o gráfico
plt.show()